In [1]:
from numpy.random import seed
seed(1)
import pickle
import numpy as np
from datetime import datetime
import os

from matplotlib import pyplot as plt
from keras import regularizers, callbacks
from keras.utils.np_utils import to_categorical
from keras.layers import (Input, Dense, Activation, ZeroPadding1D, 
BatchNormalization, Flatten, Reshape, Conv1D, MaxPooling1D, Dropout,Add, LSTM,Embedding)
from keras.initializers import glorot_normal, glorot_uniform
from keras.optimizers import Adam
from keras.models import Model, load_model
from desispec.interpolation import resample_flux

plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
#plt.rcParamsDefault['axes.prop_cycle']
plt.rcParams['font.size'] = 16
plt.rcParams['axes.grid'] = True
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rc('grid',alpha=0.3,linestyle='--')


/home/okitouni/.conda/envs/desi/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras import backend as K
def network(input_shape, learning_rate=0.0005, reg=0.0032, dropout=0.7436, seed=None):
    """ 
    Args:
    input_shape -- shape of the input spectra
    regularization_strength -- regularization factor
    dropout -- dropout rate
    seed -- seed of initializer
    Returns:
    model -- a Model() instance in Keras
    """

    X_input = Input(input_shape, name='Input_Spec')

    with K.name_scope('Conv_1'):
        X = Conv1D(filters=8, kernel_size=5, strides=1, padding='same',
                   kernel_regularizer=regularizers.l2(reg),
                   bias_initializer='zeros',
                   kernel_initializer=glorot_normal(seed))(X_input)
        X = BatchNormalization(axis=2)(X)
        X = Activation('relu')(X)
        X = MaxPooling1D(pool_size= 2)(X)

    with K.name_scope('Conv_2'):
        X = Conv1D(filters=16, kernel_size=5, strides=1, padding='same',
                   kernel_regularizer=regularizers.l2(reg),
                   bias_initializer='zeros',
                   kernel_initializer=glorot_normal(seed))(X)
        X = BatchNormalization(axis=2)(X)
        X = Activation('relu')(X)
        X = MaxPooling1D(2)(X)
    with K.name_scope('Conv_3'):
        X = Conv1D(filters=32, kernel_size=5, strides=1, padding='same',
                   kernel_regularizer=regularizers.l2(reg),
                   bias_initializer='zeros',
                   kernel_initializer=glorot_normal(seed))(X)
        X = BatchNormalization(axis=2)(X)
        X = Activation('relu')(X)
        X = MaxPooling1D(2)(X)
        
    with K.name_scope('Conv_4'):
        X = Conv1D(filters=64, kernel_size=5, strides=1, padding='same',
                   kernel_regularizer=regularizers.l2(reg),
                   bias_initializer='zeros',
                   kernel_initializer=glorot_normal(seed))(X)
        X = BatchNormalization(axis=2)(X)
        X = Activation('relu')(X)
        X = MaxPooling1D(2)(X)

        
    # FLATTEN -> FULLYCONNECTED
    with K.name_scope('Dense_Layer'):
        X = Flatten()(X)
        X = Dense(256, kernel_regularizer=regularizers.l2(reg),
                  activation='relu')(X)
        X = Dropout(rate=dropout, seed=seed)(X)
    
    with K.name_scope('Output_Layer'):
        X = Dense(1, kernel_regularizer=regularizers.l2(reg),
              activation='sigmoid',name='Output_Classes')(X)

    model = Model(inputs=X_input, outputs=X, name='SNnet')
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [3]:
model2 = network((400,1))

In [4]:
import glob
from astropy.io import fits

In [5]:
files_host = np.sort(glob.glob('/scratch/sbenzvi/desi/time-domain/bgs/hosts/*/*coadd.fits'))
flux_host = []
for f in files_host:
    h = fits.open(f)
    fl = h[1].data
    flux_host.append(fl)
fluxes_hosts = np.concatenate(flux_host)
fluxes_hosts = fluxes_hosts[fluxes_hosts.sum(axis=1)!=0]
subspec_hosts = np.mean(fluxes_hosts[:,:6000].reshape(-1,400,15),2)
maxflux = fluxes_hosts.max(axis=-1).reshape(-1,1)
minflux = fluxes_hosts.min(axis=-1).reshape(-1,1)
standarized_hosts = (subspec_hosts - minflux)/(maxflux-minflux)

In [6]:
files = np.sort(glob.glob('/scratch/sbenzvi/desi/time-domain/bgs/sne_ia/*/*coadd.fits'))
flux = []
for f in files:
    h = fits.open(f)
    f = h[1].data
    zeros = np.zeros(400)
    flux.append(f)
fluxes = np.concatenate(flux)
fluxes = fluxes[fluxes.sum(axis=1)!=0]
subspec = np.mean(fluxes[:,:6000].reshape(-1,400,15),2)
maxflux = fluxes.max(axis=-1).reshape(-1,1)
minflux = fluxes.min(axis=-1).reshape(-1,1)
standarized = (subspec - minflux)/(maxflux-minflux)

In [7]:
x_train = np.concatenate([standarized,standarized_hosts]).reshape(-1,400,1)
y_train = np.concatenate([np.zeros(standarized.shape[0]),np.ones(standarized_hosts.shape[0])])
permute = np.random.permutation(y_train.shape[0])

In [8]:
hist = model2.fit(x_train[permute][:30000],y_train[permute][:30000],batch_size=64,epochs=50,
                  validation_split=0.1,shuffle=True)

Train on 27000 samples, validate on 3000 samples
Epoch 1/50
27000/27000 [==============================] - 47s 2ms/step - loss: 1.5754 - acc: 0.5574 - val_loss: 1.1333 - val_acc: 0.5753
Epoch 2/50
27000/27000 [==============================] - 43s 2ms/step - loss: 0.9455 - acc: 0.6300 - val_loss: 0.8659 - val_acc: 0.5967
Epoch 3/50
27000/27000 [==============================] - 43s 2ms/step - loss: 0.6768 - acc: 0.7548 - val_loss: 1.3656 - val_acc: 0.4483
Epoch 4/50
27000/27000 [==============================] - 45s 2ms/step - loss: 0.5765 - acc: 0.7947 - val_loss: 0.6615 - val_acc: 0.7027
Epoch 5/50
27000/27000 [==============================] - 42s 2ms/step - loss: 0.5258 - acc: 0.8133 - val_loss: 0.9394 - val_acc: 0.5907
Epoch 6/50
27000/27000 [==============================] - 42s 2ms/step - loss: 0.4909 - acc: 0.8306 - val_loss: 2.0243 - val_acc: 0.4473
Epoch 7/50
27000/27000 [==============================] - 43s 2ms/step - loss: 0.4607 - acc: 0.8423 - val_loss: 0.9979 - val_acc:

In [ ]:
hist2 = model2.fit(x_train[permute][:30000],y_train[permute][:30000],batch_size=4,epochs=100,
                  validation_split=0.1,shuffle=True)

Train on 27000 samples, validate on 3000 samples
Epoch 1/100
27000/27000 [==============================] - 142s 5ms/step - loss: 0.5152 - acc: 0.8145 - val_loss: 0.4730 - val_acc: 0.8400
Epoch 2/100
27000/27000 [==============================] - 143s 5ms/step - loss: 0.4593 - acc: 0.8448 - val_loss: 0.4879 - val_acc: 0.8093
Epoch 3/100
27000/27000 [==============================] - 143s 5ms/step - loss: 0.4476 - acc: 0.8493 - val_loss: 0.4673 - val_acc: 0.8300
Epoch 4/100
27000/27000 [==============================] - 141s 5ms/step - loss: 0.4382 - acc: 0.8495 - val_loss: 0.8181 - val_acc: 0.6293
Epoch 5/100
27000/27000 [==============================] - 143s 5ms/step - loss: 0.4300 - acc: 0.8540 - val_loss: 0.6477 - val_acc: 0.7097
Epoch 6/100
27000/27000 [==============================] - 144s 5ms/step - loss: 0.4245 - acc: 0.8548 - val_loss: 0.4767 - val_acc: 0.8240
Epoch 7/100
27000/27000 [==============================] - 141s 5ms/step - loss: 0.4175 - acc: 0.8587 - val_loss: 0.4

In [ ]:
hist = model2.fit(x_train[permute],y_train[permute],batch_size=64,epochs=50,
                  validation_split=0.1,shuffle=True)

Train on 161244 samples, validate on 17917 samples
Epoch 1/50
161244/161244 [==============================] - 206s 1ms/step - loss: 0.7901 - acc: 0.7280 - val_loss: 0.9118 - val_acc: 0.6655
Epoch 2/50
161244/161244 [==============================] - 203s 1ms/step - loss: 0.4447 - acc: 0.8437 - val_loss: 0.7148 - val_acc: 0.7388
Epoch 3/50
161244/161244 [==============================] - 208s 1ms/step - loss: 0.3960 - acc: 0.8624 - val_loss: 0.5286 - val_acc: 0.7645
Epoch 4/50
161244/161244 [==============================] - 198s 1ms/step - loss: 0.3775 - acc: 0.8681 - val_loss: 0.7683 - val_acc: 0.6891
Epoch 5/50
161244/161244 [==============================] - 205s 1ms/step - loss: 0.3653 - acc: 0.8735 - val_loss: 0.4605 - val_acc: 0.8005
Epoch 6/50
161244/161244 [==============================] - 212s 1ms/step - loss: 0.3557 - acc: 0.8761 - val_loss: 1.2020 - val_acc: 0.6592
Epoch 7/50
161244/161244 [==============================] - 203s 1ms/step - loss: 0.3466 - acc: 0.8792 - val_